### 데이터 불러오기

In [1]:
!pip3 install pmdarima

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
import seaborn as sns

os.chdir('/content/drive/MyDrive/신입기수 프로젝트/Dataset')

In [4]:
df = pd.read_csv('rawdata.csv')

In [5]:
df_arima = df[['날짜', '청담대교_수위', '잠수교_수위', '한강대교_수위', '행주대교_유량', '행주대교_수위']]
df_arima

,날짜,청담대교_수위,잠수교_수위,한강대교_수위,행주대교_유량,행주대교_수위
0,2012-05-01 00:00,310.7,300.2,290.0,540.18,275.3
1,2012-05-01 00:10,314.7,300.2,290.0,540.18,275.3
2,2012-05-01 00:20,313.7,301.2,290.0,540.18,275.3
3,2012-05-01 00:30,311.7,301.2,290.0,552.17,276.3
4,2012-05-01 00:40,311.7,301.2,291.0,564.29,277.3
...,...,...,...,...,...,...
276331,2022-07-18 23:10,0.0,0.0,0.0,974.40,0.0
276332,2022-07-18 23:20,0.0,0.0,0.0,1006.88,0.0
276333,2022-07-18 23:30,0.0,0.0,0.0,1039.90,0.0
276334,2022-07-18 23:40,0.0,0.0,0.0,1073.46,0.0


In [6]:
df_arima.index = list(df_arima.index)
df_arima.index

Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            276326, 276327, 276328, 276329, 276330, 276331, 276332, 276333,
            276334, 276335],
           dtype='int64', length=276336)

### 결측치 확인하기

In [7]:
# 결측치 처리할 컬럼

col = ['청담대교_수위', '잠수교_수위', '한강대교_수위', '행주대교_유량', '행주대교_수위']

In [8]:
# 결측치 인덱스 확인 (df_arima[i].isnull().index는 안되고 다음과 같이 앞에 df_arima를 한번 더 붙여주면 됨!)

for i in col:
  print(df_arima[df_arima[i].isnull()].index)
  print('------------------------')

Int64Index([140613, 140614, 140615, 140616, 140617, 140618, 140619, 140620,
            140621, 140622, 140623, 140624, 140625, 140626, 140627, 140628,
            140629, 140630, 140631, 140632, 140633, 140634, 140635, 140636,
            140637, 140638, 140639, 140640, 140641, 140642, 140643, 140644,
            140645, 140646, 140647, 140648, 140649, 140650, 140651, 140652,
            140653, 140654, 140655, 140656, 140657, 140658, 140659, 140660,
            140661, 140662, 140663, 140664, 140665, 140666, 140667, 140668,
            140669, 140670, 236080],
           dtype='int64')
------------------------
Int64Index([140613, 140614, 140615, 140616, 140617, 140618, 140619, 140620,
            140621, 140622, 140623, 140624, 140625, 140626, 140627, 140628,
            140629, 140630, 140631, 140632, 140633, 140634, 140635, 140636,
            140637, 140638, 140639, 140640, 140641, 140642, 140643, 140644,
            140645, 140646, 140647, 140648, 140649, 140650, 140651, 140652,


### 연속-불연속 결측치 리스트 만들기

In [9]:
nanIndex = df_arima[df_arima[col[0]].isnull()].index
nanIndex

Int64Index([140613, 140614, 140615, 140616, 140617, 140618, 140619, 140620,
            140621, 140622, 140623, 140624, 140625, 140626, 140627, 140628,
            140629, 140630, 140631, 140632, 140633, 140634, 140635, 140636,
            140637, 140638, 140639, 140640, 140641, 140642, 140643, 140644,
            140645, 140646, 140647, 140648, 140649, 140650, 140651, 140652,
            140653, 140654, 140655, 140656, 140657, 140658, 140659, 140660,
            140661, 140662, 140663, 140664, 140665, 140666, 140667, 140668,
            140669, 140670, 236080],
           dtype='int64')

In [10]:
unconsecutive = []
consecutive = []

i = 0
while i < len(nanIndex):
    if nanIndex[i]+1 in nanIndex: # 다음 거가 nanIndex 안에 있으면 다음 거까지 consecutive에 추가
        consecutive.extend([nanIndex[i], nanIndex[i+1]])
        try:
            i += 2
        except: #i가 len(nanIndex2) - 1
            break
    elif nanIndex[i]-1 in nanIndex: #다음 거는 nanIndex 안에 없지만 consecutive로 들어가야 하는 경우(반복 구간의 마지막 원소)
        consecutive.append(nanIndex[i])
        i += 1
    else:
        unconsecutive.append(nanIndex[i])
        i += 1

In [11]:
print(len(consecutive), consecutive)
print(len(unconsecutive), unconsecutive)

58 [140613, 140614, 140615, 140616, 140617, 140618, 140619, 140620, 140621, 140622, 140623, 140624, 140625, 140626, 140627, 140628, 140629, 140630, 140631, 140632, 140633, 140634, 140635, 140636, 140637, 140638, 140639, 140640, 140641, 140642, 140643, 140644, 140645, 140646, 140647, 140648, 140649, 140650, 140651, 140652, 140653, 140654, 140655, 140656, 140657, 140658, 140659, 140660, 140661, 140662, 140663, 140664, 140665, 140666, 140667, 140668, 140669, 140670]
1 [236080]


### 보간처리

In [12]:
# 보간처리를 위해 앞뒤 인덱스를 포함한 unconsecutive2 인덱스
unconsecutive2 = sum([[i-1, i, i+1] for i in unconsecutive], [])
# sum(list, []) : to unfold the list

In [13]:
# 데이터프레임 분리
df_intp = df_arima.copy()
df_intp = df_intp.iloc[unconsecutive2, :]
df_intp

,날짜,청담대교_수위,잠수교_수위,한강대교_수위,행주대교_유량,행주대교_수위
236079,2020-10-15 10:30,317.7,330.2,312.0,762.23,292.3
236080,2020-10-15 10:40,NaN,NaN,NaN,NaN,NaN
236081,2020-10-15 10:50,314.7,326.2,308.0,734.09,290.3


In [14]:
# 보간
df_intp.interpolate(inplace=True)
df_intp   # 보간 결과 확인

,날짜,청담대교_수위,잠수교_수위,한강대교_수위,행주대교_유량,행주대교_수위
236079,2020-10-15 10:30,317.7,330.2,312.0,762.23,292.3
236080,2020-10-15 10:40,316.2,328.2,310.0,748.16,291.3
236081,2020-10-15 10:50,314.7,326.2,308.0,734.09,290.3


In [15]:
# 보간처리한 row들을 df_arima에 합치기
for i in col:
  df_arima.loc[unconsecutive2, [i]] = df_intp.loc[:, [i]]

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1884: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, val, pi)


In [16]:
# 남은 결측치 개수가 SARIMA로 채워야 할 결측치 개수(len(consecutive))와 같은지 확인
df_arima.isnull().sum() == len(consecutive)

날짜         False
청담대교_수위     True
잠수교_수위      True
한강대교_수위     True
행주대교_유량     True
행주대교_수위     True
dtype: bool

### Arima

In [17]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pmdarima as pm

In [18]:
n = 432

for i in col:
    start = consecutive[0]
    model = pm.auto_arima(df_arima[i][start-n:start], seasonal = True)
    df_arima[i].loc[consecutive] = model.predict(len(consecutive)).values

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  data=self.data,
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  data=self.data,
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pa

In [19]:
df_arima.isnull().sum()

날짜         0
청담대교_수위    0
잠수교_수위     0
한강대교_수위    0
행주대교_유량    0
행주대교_수위    0
dtype: int64

In [21]:
df_arima.to_csv('/content/drive/MyDrive/신입기수 프로젝트/Dataset/0821_기타결측치_Arima.csv', index = False, encoding = 'cp949')